# Exploring 21st Century Data

The purpose of this notebook is to identify what data is available from a set of data from 21st Century Equipment. While it is different than the data we have in Anvil this exploration will be used to identify ways we can measure/predict customer buying cycles and potentially customer segments.

## Data Overview

- **Unique Product Groups**: 45
- **Total Sales**: $3,984,730,575.96
- **Total units sold**: 79,218
- **Earliest sale**: 2011-01-03

It's highly likely that quite a few of these transactions are mislabeled from a Product Group perspective. To address this we may need to do make/model matching from our database to verify or update product grouping. For discovery work we can try ignoring things less than $1000 as these are likely not something that we would want to generate leads for. 

From a customer perspective we ignore the following:
- `CONVERSION CUSTOMER` : These are customers from a previous dealerships and didn't have an account associated with 21st century. This will be a common thing we will want to clean from dealerships in order to do our analysis.
- `MV EQUIP CONVERSION CUST` : Same as above
- `21ST CENTURY LEASING LLC` : Seems to have sold to a subsidiary and will have to be something we look at. Want to ignore for evaluating customer buying cycles
- `Big Iron`: Equipment sent to auction. While useful in some contexts, not an actual customer


In [32]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import ticker

## Cleaning Data

Making dates into date objects and cleaning up the numbers. All the money fields have dollar signs/commas and the percentage fields have the percentage signs. These are cleaned up to be floating values and the percentages given in absolute terms.

In [ ]:
df = pd.read_csv("../data/21st-century-data.csv")
# split Account into account name and account number by splitting on "--"
df["Customer Invoice Date"] = pd.to_datetime(df["Customer Invoice Date"])
df["Inventory Date  Date"] = pd.to_datetime(df["Inventory Date  Date"])
df["Inventory Year"] = df["Inventory Date  Date"].dt.year
df["Inventory Month"] = df["Inventory Date  Date"].dt.month
df["Days In Inventory "] = df["Days In Inventory "].astype(int)
df["Machine Age"] = df["Machine Age"].astype(int)
df["Engine Hours"] = df["Engine Hours"].astype(int)
df["Month Number "] = df["Month Number "].astype(int)
df["Buying Year "] = df["Buying Year "].astype(int)
df["Invoice Amount"] = df["Invoice Amount"].str.replace("$", "").str.replace(",", "").astype(float)
df["Customer Invoice Amount"] = df["Customer Invoice Amount"].str.replace("$", "").str.replace(",", "").astype(float)
df["MDP (Approx)"] = df["MDP (Approx)"].str.replace("$", "").str.replace(",", "").astype(float)
df["Equipment Cost"] = df["Equipment Cost"].str.replace("$", "").str.replace(",", "").astype(float)
df["Net Trade Amount"] = df["Net Trade Amount"].str.replace("$", "").str.replace(",", "").astype(float)
df["Invoice as a Percent of MDP"] = df["Invoice as a Percent of MDP"].str.replace("%", "").replace("#DIV/0!", "0").apply(lambda x: float(x) / 100)
# Margin Dollars has a $ sign and uses ( ) to denote negative values
df["Margin Dollars "] = df["Margin Dollars "].str.replace("$", "").str.replace("(", "-").str.replace(")", "").str.replace(",", "").astype(float)
df["Change for Net Trade to Equipment Cost "] = df["Change for Net Trade to Equipment Cost "].str.replace("$", "").str.replace("(", "-").str.replace(")", "").str.replace(",", "").astype(float)
df["Margin Percent "] = df["Margin Percent "].str.replace("%", "").replace("#DIV/0!", "0").apply(lambda x: float(x) / 100)
df["account_name"] = df["Account"].apply(lambda x: x.split("--")[0]).str.strip()
df["account_number"] = df["Account"].apply(lambda x: x.split("--")[1]).str.strip()
# Update column names with trailing spaces to remove them
df = df.rename(columns={col: col.strip() for col in df.columns})
df.head()

In [83]:
df.to_csv("../data/clean-21st-century-data.csv", index=False)

In [ ]:
print(df.columns)

In [ ]:
num_product_groups = len(df["Product Group"].unique())
print(f"There are {num_product_groups} unique product groups")

In [ ]:
total_sales = df["Invoice Amount"].sum()
total_units = len(df)
print(f"Total sales: ${total_sales:,.2f}")
print(f"Total units sold: {total_units:,}")

In [ ]:
earliest_sale = df["Customer Invoice Date"].min()
print(f"Earliest sale: {earliest_sale}")

In [ ]:
latest_sale = df["Customer Invoice Date"].max()
print(f"Latest sale: {latest_sale}")

In [ ]:
df.describe()

In [ ]:
df.dropna(subset=["Product Group"]).loc[df.dropna(subset=["Product Group"])["Product Group"].str.contains("ATTACHMENT")].describe()

In [ ]:
df["Product Group"].value_counts().nlargest(15).sort_values().plot(kind="barh", figsize=(10, 6))
plt.title("Top 15 Product Groups by Count")

In [ ]:
def millions(x, pos):
    if x >= 1e9:
        return f"${x * 1e-9:,.3f}B"
    return f"${x * 1e-6:,.0f}M"

invoice_amount_by_product_group = df[["Product Group", "Invoice Amount"]].groupby("Product Group").sum().sort_values(by="Invoice Amount", ascending=False).head(15).reset_index()
invoice_amount_by_product_group.head()
fig = plt.figure(figsize=(10, 6))
ax = plt.gca()
sns.barplot(x="Invoice Amount", y="Product Group", data=invoice_amount_by_product_group, ax=ax)
# correct yticks to be in millions of $
ax.xaxis.set_major_formatter(ticker.FuncFormatter(millions))
plt.title("Top 15 Total Invoice Amount by Product Group")
plt.show()

In [ ]:
df[["Product Group", "Invoice Amount"]].groupby("Product Group").sum().sort_values(by="Invoice Amount", ascending=False).head(15).iloc[::-1].plot(kind="barh", figsize=(10, 6), legend=False )
plt.title("Top 15 Product Groups by Invoice Amount")

In [ ]:
df[["Product Group", "Invoice Amount"]].groupby("Product Group").mean().sort_values(by="Invoice Amount", ascending=False).head(15).iloc[::-1].plot(kind="barh", figsize=(10, 6), legend=False )
plt.title("Top 15 Product Groups by Average Invoice Amount")

In [ ]:
df[["Product Group", "Invoice Amount"]].groupby("Product Group").mean().sort_values(by="Invoice Amount", ascending=True).head(15).iloc[::-1].plot(kind="barh", figsize=(10, 6), legend=False )
plt.title("Bottom 15 Product Groups by Average Invoice Amount")

In [ ]:
# create a 2x1 grid of plots
fig, ax = plt.subplots(2, 1, figsize=(10, 12))
# plot the top 15 product groups by count
df[["Product Group", "Invoice Amount"]].groupby("Product Group").count().sort_values(by="Invoice Amount", ascending=False).head(15).iloc[::-1].plot(kind="barh", ax=ax[0], legend=False)
# plot the top 15 product groups by count where the invoice amount is greater than 1000
df[df["Invoice Amount"] > 1000][["Product Group", "Invoice Amount"]].groupby("Product Group").count().sort_values(by="Invoice Amount", ascending=False).head(15).iloc[::-1].plot(kind="barh", ax=ax[1], legend=False)
ax[0].set_title("Top 15 Product Groups by Total Units Sold")
ax[1].set_title("Top 15 Product Groups by Total Units Sold where Invoice Amount > $1000")

In [ ]:
attachment_product_groups = df.dropna(subset=["Product Group"]).loc[df.dropna(subset=["Product Group"])["Product Group"].str.contains("ATTACHMENT")]["Product Group"].unique().tolist()
attachment_df = df.loc[df["Product Group"].isin(attachment_product_groups)]
sns.violinplot(x="Product Group", y="Invoice Amount", data=attachment_df)
plt.xticks(rotation=90)

In [ ]:
df.loc[(df["Product Group"] == "TURF ATTACHMENTS") & (df["Invoice Amount"] > 100000)]

In [ ]:
df.loc[(df["Product Group"] == "TRACTOR ATTACHMENTS") & (df["Invoice Amount"] > 100000)]

In [ ]:
attachment_grouped_df = attachment_df[["Product Group", "Invoice Amount"]].groupby("Product Group").agg(["count", "sum", "mean", "median", "std"]).reset_index()
attachment_grouped_df.columns = [np.arange(0,attachment_grouped_df.shape[1])]
attachment_grouped_df.columns = ["Product Group", "Count", "Sum", "Mean", "Median", "Std"]
attachment_grouped_df.head(10)

In [74]:
# find all of the units that are more than 2 standard deviations above the mean
attachment_df = attachment_df.merge(attachment_grouped_df, on="Product Group")

In [ ]:
attachment_df["z_score"] = (attachment_df["Invoice Amount"] - attachment_df["Mean"]) / attachment_df["Std"]
attachment_df.loc[attachment_df["z_score"] > 2]

In [ ]:
sns.violinplot(x="Product Group", y="Invoice Amount", data=attachment_df.loc[attachment_df["z_score"] < 2])
plt.xticks(rotation=90)

In [ ]:
df[["account_name", "Customer Invoice Amount"]].groupby("account_name").mean().sort_values(by="Customer Invoice Amount", ascending=False)

In [ ]:
# get total sales for the top 1% of accounts ignoring Conversion Customers
bad_account_names = ["CONVERSION CUSTOMER", "MV EQUIP CONVERSION CUST", "21ST CENTURY LEASING LLC", "BIG IRON"]
top_1_percent_accounts = df.loc[~df["account_name"].isin(bad_account_names)][["account_name", "Invoice Amount"]].groupby("account_name").sum().sort_values(by="Invoice Amount", ascending=False).head(int(len(df["account_name"].unique()) * 0.01))
top_1_percent_accounts.head()

In [ ]:
top_1_percent_accounts.sum()

In [ ]:
top_1_percent_accounts.sum() / 3984370576

Most of the highest grossing accounts are:
1. A bucket of customers who aren't matched to the parent company in an acquisition
2. An agriculture service company (e.g. for harvesting)
3. A leasing arm of the company
4. Large corporations or conglomorates

In [ ]:
df[["account_name", "Invoice Amount"]].groupby("account_name").sum().sort_values(by="Invoice Amount", ascending=False).head(15).iloc[::-1].plot(kind="barh", figsize=(10, 6), legend=False )

In [ ]:
invoice_amount_by_customer = df[["account_name", "Invoice Amount"]].groupby("account_name").sum().sort_values(by="Invoice Amount", ascending=False).head(15).reset_index()
fig = plt.figure(figsize=(10, 6))
ax = plt.gca()
sns.barplot(x="Invoice Amount", y="account_name", data=invoice_amount_by_customer, ax=ax)
# correct yticks to be in millions of $
ax.xaxis.set_major_formatter(ticker.FuncFormatter(millions))
plt.title("Top 15 Total Invoice Amount by Customer")
plt.show()

In [ ]:
df[["account_name", "Invoice Amount"]].groupby("account_name").count().sort_values(by="Invoice Amount", ascending=False).head(15).iloc[::-1].plot(kind="barh", figsize=(10, 6), legend=False )

In [ ]:
df["New/Used"].value_counts().plot(kind="bar")

In [ ]:
df["New/Used"].describe()

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = plt.gca()
sns.barplot(x="New/Used", y="Invoice Amount", data=df.groupby("New/Used")["Invoice Amount"].sum().reset_index(), ax=ax)
# correct yticks to be in millions of $
ax.yaxis.set_major_formatter(ticker.FuncFormatter(millions))
plt.title("Total Invoice Amount by New/Used")
plt.show()

In [ ]:
df.groupby("New/Used")["Invoice Amount"].mean().plot(kind="bar")

In [ ]:
sns.barplot(x="Buying Year", y="Invoice Amount", data=df)

In [ ]:
sns.barplot(x="Buying Year", y="Invoice Amount", hue="New/Used", data=df)
plt.title("Average Invoice Amount by Buying Year")

## Purchase Frequency

Look at when customers purchase equipment by year/month and how many purchases are repeat

In [ ]:
frequency_year_df = df.copy()
frequency_year_df = frequency_year_df.groupby(["Product Group", "Buying Year"]).agg({"Invoice Amount": "sum", "account_name": "count"}).reset_index()
frequency_year_df.columns = ["Product Group", "Buying Year", "Total Invoices", "Total Sales"]
frequency_year_df.head()

In [ ]:
sns.barplot(x="Buying Year", y="Total Invoices", data=frequency_year_df)

In [23]:
tractor_product_groups = frequency_year_df.dropna(subset=["Product Group"]).loc[frequency_year_df.dropna(subset=["Product Group"])["Product Group"].str.contains("TRACTOR")]["Product Group"].unique().tolist()

In [ ]:
def millions(x, pos):
    return f"${x * 1e-6:.0f}M"


fig = plt.figure(figsize=(10, 6))
ax = plt.gca()
sns.lineplot(x="Buying Year", y="Total Invoices", hue="Product Group", data=frequency_year_df.loc[frequency_year_df["Product Group"].isin(tractor_product_groups)])
# correct yticks to be in millions of $
ax.yaxis.set_major_formatter(ticker.FuncFormatter(millions))
plt.title("Total Invoice Amount by Product Group and Buying Year - Tractors")
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = plt.gca()
sns.lineplot(x="Buying Year", y="Total Sales", hue="Product Group", data=frequency_year_df.loc[frequency_year_df["Product Group"].isin(tractor_product_groups)])
plt.title("Total Sales by Product Group and Buying Year - Tractors")
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = plt.gca()
sns.lineplot(x="Buying Year", y="Total Invoices", hue="Product Group", data=frequency_year_df.loc[frequency_year_df["Product Group"] == "COMBINES"])
plt.title("Total Invoice Amount by Product Group and Buying Year - Combines")
ax.yaxis.set_major_formatter(ticker.FuncFormatter(millions))
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = plt.gca()
sns.lineplot(x="Buying Year", y="Total Sales", hue="Product Group", data=frequency_year_df.loc[frequency_year_df["Product Group"] == "COMBINES"])
plt.title("Total Sales by Product Group and Buying Year - Combines")
plt.show()

## Create Buying Cycle Dataset

This dataset will look at customers buying larger pieces of equipment (>$1,000 and not an attachment). Multiple purchases will be required and we clean up data like machine age and engine hours (if applicable). Also remove the non-customer accounts.

In [ ]:
bad_account_names = ["CONVERSION CUSTOMER", "MV EQUIP CONVERSION CUST", "21ST CENTURY LEASING LLC", "BIG IRON"]
cycle_df = df.copy()
cycle_df = cycle_df.loc[~cycle_df["account_name"].isin(bad_account_names)]
cycle_df = cycle_df.loc[cycle_df["Customer Invoice Amount"] > 1000]
cycle_df = cycle_df.loc[~cycle_df["Product Group"].isna()]
cycle_df = cycle_df.loc[~cycle_df["Product Group"].str.contains("ATTACHMENT")]
cycle_df["combined_product_group"] = cycle_df["Product Group"].apply(lambda x: "LARGE TRACTOR" if x in ["TRACTOR-ROW CROP", "TRACTOR-4WD", "TRACTOR-TRACK"] else x)
# Now only include records where an account_name appears at least twice for each combined_product_group
cycle_df = cycle_df.groupby(["account_name", "combined_product_group"]).filter(lambda x: len(x) > 1)
# calculate year from previous purchase for same account name and product group
cycle_df.sort_values(by=["account_name", "combined_product_group", "Customer Invoice Date"], ascending = [True, True, True],inplace=True)
cycle_df["previous_purchase_year"] = cycle_df.groupby(["account_name", "combined_product_group"])["Buying Year"].shift(1)
cycle_df["year_from_previous_purchase"] = cycle_df["Buying Year"] - cycle_df["previous_purchase_year"]
cycle_df = cycle_df[["account_name", "account_number", "Stock Number", "Machine Serial Number", "Engine Hours", "Unit Hours 2",  "combined_product_group", "Make", "Model", "New/Used", "Model Year", "Machine Age", "Invoice Amount", "Customer Invoice Date", "Month Number", "Buying Year", "year_from_previous_purchase"]]
cycle_df.head()

In [13]:
cycle_df.to_csv("../data/cycle-21st-century-data.csv", index=False)

In [ ]:
cycle_df["combined_product_group"].value_counts()

In [ ]:
cycle_df.groupby("combined_product_group")["year_from_previous_purchase"].mean().sort_values(ascending=False)

In [ ]:
cycle_df.account_name.nunique()